# Import Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/User/Documents/le wagon project/coronanet_release_allvars.csv', encoding='mac_roman')

In [3]:
for col in df.columns: 
    print(col) 

Unnamed: 0
record_id
policy_id
entry_type
correct_type
update_type
update_level
description
date_announced
date_start
date_end
country
ISO_A3
ISO_A2
init_country_level
domestic_policy
province
city
type
type_sub_cat
type_text
institution_status
target_country
target_geog_level
target_region
target_province
target_city
target_other
target_who_what
target_direction
travel_mechanism
compliance
enforcer
index_high_est
index_med_est
index_low_est
index_country_rank
link
date_updated
recorded_date
confirmed_cases
deaths
recovered
new_tests
total_tests
total_tests_per_thousand
tests_units
Rank_FP
Score_FP
state_IDC
muni_IDC
dispersive_IDC
constraining_IDC
inclusive_IDC
sfi_SFI
ti_cpi_TI
pop_WDI_PW
gdp_WDI_PW
gdppc_WDI_PW
growth_WDI_PW
lnpop_WDI_PW
lngdp_WDI_PW
lngdppc_WDI_PW
disap_FA
polpris_FA
latentmean_FA
transparencyindex_HR
EmigrantStock_EMS
v2x_polyarchy_VDEM


# keep the important columns

In [4]:
 def tristans_columns(self):
        df = self.copy()
        df['date_announced'] = pd.to_datetime(df['date_announced'])
        df['date_start'] = pd.to_datetime(df['date_start'])
        df['date_end'] = pd.to_datetime(df['date_end'])
        df['date_announced'].fillna(df['date_start'], inplace = True)
        df['date_start'].clip(lower=df['date_announced'], inplace=True)
        df_select = df[['policy_id', 'date_start', 'country',
           'type', 'type_sub_cat','description', 'ISO_A3','compliance']]
        df_select.drop_duplicates(subset=['policy_id'], inplace=True)
        return df_select

In [5]:
df= tristans_columns(df)

c:\users\user\.venvs\lewagon\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
for col in df.columns: 
    print(col) 

policy_id
date_start
country
type
type_sub_cat
description
ISO_A3
compliance


In [7]:
df.isnull().sum(axis = 0)

policy_id           0
date_start          0
country             0
type                0
type_sub_cat    10257
description         0
ISO_A3            133
compliance        125
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.isnull().sum(axis = 0)

policy_id       0
date_start      0
country         0
type            0
type_sub_cat    0
description     0
ISO_A3          0
compliance      0
dtype: int64

In [10]:
df

,policy_id,date_start,country,type,type_sub_cat,description,ISO_A3,compliance
4,6721435,2020-03-09,Afghanistan,Closure and Regulation of Schools,Preschool or childcare facilities (generally f...,"""All schools in Herat ñ including temporarily ...",AFG,Mandatory (Unspecified/Implied)
7,6018486,2020-03-11,Afghanistan,Closure and Regulation of Schools,Primary Schools (generally for children ages 1...,"In parts of northern Samangan province, school...",AFG,Mandatory (Unspecified/Implied)
10,3558044,2020-03-14,Afghanistan,Closure and Regulation of Schools,Preschool or childcare facilities (generally f...,"""On 14 March, the Government of Afghanistan an...",AFG,Mandatory (Unspecified/Implied)
18,2231982,2020-04-18,Afghanistan,Closure and Regulation of Schools,Primary Schools (generally for children ages 1...,"April 11, Afghanistan's ""President Ghani says ...",AFG,Mandatory (Unspecified/Implied)
21,1212064,2020-06-03,Afghanistan,Closure and Regulation of Schools,Preschool or childcare facilities (generally f...,"April 14 ""Afghanistan on Saturday announced th...",AFG,Mandatory (Unspecified/Implied)
...,...,...,...,...,...,...,...,...
45629,6143876,2020-10-02,Zimbabwe,Restriction and Regulation of Government Services,Election procedures (e.g. mail-in voting),"The Zimbabwean minister for health, who is als...",ZWE,Mandatory (Unspecified/Implied)
45630,2531192,2020-03-02,Zimbabwe,Restrictions of Mass Gatherings,Prison population reduced (e.g. early release ...,A presidential amnesty was announced by the Zi...,ZWE,Voluntary/Recommended but No Penalties
45631,2574281,2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,Postponement of an annually recurring event,"In Zimbabwe, the 40th independence anniversary...",ZWE,Mandatory (Unspecified/Implied)
45632,6689905,2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,Postponement of an annually recurring event,"In Zimbabwe, the Zimbabwe International Trade ...",ZWE,Mandatory (Unspecified/Implied)


# Tokenization and lemmatization

In [11]:
import numpy as np
import pandas as pd
import datetime
import pyreadr
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
def clean(x):
    x = x.lower()
    x = ''.join([letter for letter in x if not letter.isdigit()])
    for p in string.punctuation:
        x = x.replace(p, " ")
    stops = set(stopwords.words("english"))
    tokens = word_tokenize(x)
    tokens = ' '.join([token for token in tokens if token not in stops])
    return tokens

In [13]:
df['clean_description'] = df.description.apply(clean)

In [14]:
df['clean_description']

4        schools herat ñ including temporarily learning...
7        parts northern samangan province schools close...
10       march government afghanistan announced schools...
18       april afghanistan president ghani says schools...
21       april afghanistan saturday announced closure e...
                               ...                        
45629    zimbabwean minister health also vice president...
45630    presidential amnesty announced zimbabwe govern...
45631    zimbabwe th independence anniversary celebrati...
45632    zimbabwe zimbabwe international trade fair zit...
45639    zimbabwe minister information publicity announ...
Name: clean_description, Length: 14555, dtype: object

In [15]:
def lemma(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [16]:
df['clean_description'] = lemma(df['clean_description'])

In [65]:
df['clean_description']

4        schools herat ñ including temporarily learning...
7        parts northern samangan province schools close...
10       march government afghanistan announced schools...
18       april afghanistan president ghani says schools...
21       april afghanistan saturday announced closure e...
                               ...                        
45629    zimbabwean minister health also vice president...
45630    presidential amnesty announced zimbabwe govern...
45631    zimbabwe th independence anniversary celebrati...
45632    zimbabwe zimbabwe international trade fair zit...
45639    zimbabwe minister information publicity announ...
Name: clean_description, Length: 14555, dtype: object

# Now lets encode my subcat

In [34]:
df[['type_sub_cat']]

,type_sub_cat
4,Preschool or childcare facilities (generally f...
7,Primary Schools (generally for children ages 1...
10,Preschool or childcare facilities (generally f...
18,Primary Schools (generally for children ages 1...
21,Preschool or childcare facilities (generally f...
...,...
45629,Election procedures (e.g. mail-in voting)
45630,Prison population reduced (e.g. early release ...
45631,Postponement of an annually recurring event
45632,Postponement of an annually recurring event


In [46]:
temp = df['type_sub_cat'].value_counts()

In [54]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit_transform(list(pd.DataFrame(temp).reset_index()['index']))

array([ 14,  85,  39,  62,  46,  77,  83,  47,   1,  63,  42,  48,  49,
        35,  29,  21,  96,  26,  33,  11,  53,   3,   9, 108,  15,  66,
        91,  45,  54,  84,  31,  18,  89,  90,  10,  28,  30,  51,  94,
       102,  58, 104,  50,  52,  73,   6,  44,  86,   4,  97,  40,   2,
        23,  57,  93,  60,  87,  25, 101,  68,  61,  20,  55,  67,  22,
        17,  78,  75,  27,  13,  69,  65, 103,   5,  95,  64,  16,  56,
        74,  24,  80,  43,  36,  72, 100,  98,  12,   7,  34, 111, 112,
       109,  19,  79,  59,  71,   0,   8,  92, 110,  99,  88,  41,  81,
        82, 105, 107,  37,  76,  70, 106,  38,  32, 113], dtype=int64)

In [58]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['type_sub_cat'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))


In [60]:
df['encoded_sub-cat']= df['type_sub_cat'].map(le_name_mapping)

# Now we can run a Naive Bayes model and see its score

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_description']) 
y = df['encoded_sub-cat']

In [94]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()

nb_model.fit(X,y)

nb_model.score(X,y)

0.35829611817244933

# lets double check to see if the model can predict the right sub-cat

In [96]:
nb_model.predict_proba(X)

array([[3.60466322e-05, 8.82186258e-03, 3.54663016e-04, ...,
        7.62490570e-05, 6.78235381e-05, 3.15803873e-06],
       [1.92949918e-04, 2.77995996e-02, 1.40532509e-03, ...,
        3.75268146e-04, 2.89169607e-04, 1.65056906e-05],
       [2.02962741e-05, 1.38119801e-02, 2.51104295e-04, ...,
        4.49931020e-05, 4.61030363e-05, 1.75936357e-06],
       ...,
       [1.04494145e-04, 2.78198305e-02, 2.52586981e-03, ...,
        1.74110145e-04, 1.69154319e-04, 8.42158130e-06],
       [1.41895129e-04, 2.47933074e-02, 2.03887215e-03, ...,
        2.54752156e-04, 2.64982116e-04, 1.29779642e-05],
       [2.10340691e-04, 4.16890394e-02, 4.20364757e-03, ...,
        3.84779920e-04, 3.38334194e-04, 1.92912823e-05]])

## I'll take the 46th line 

In [97]:
sorted(nb_model.predict_proba(X)[45], reverse = True)

[0.15989831140973307,
 0.11537906852487613,
 0.07773879719108188,
 0.047567249229201775,
 0.036041359970286464,
 0.03193329455167043,
 0.03079154412600258,
 0.026232268174908858,
 0.026161094950477328,
 0.025702800038971844,
 0.02321952896309826,
 0.023130324245042515,
 0.021567312568207566,
 0.021539871873981615,
 0.020758489684831835,
 0.018401497210996837,
 0.015054387592870916,
 0.0136901847128596,
 0.011705862742756395,
 0.01057148311065089,
 0.009758642498250761,
 0.00967719624250761,
 0.009265005791541598,
 0.009048776793710958,
 0.008463990933431783,
 0.00820418738884154,
 0.008008009645902845,
 0.007680305854100941,
 0.007496495683366788,
 0.007421893853197731,
 0.007380693880548705,
 0.007229982917480353,
 0.006803790973640585,
 0.006588278707354878,
 0.006115303383351933,
 0.005699786877303125,
 0.005125872130662444,
 0.005050072906468717,
 0.004850754999517997,
 0.0048349485960608235,
 0.004782022030556272,
 0.00462548158259348,
 0.004624791242031542,
 0.004228915834918178,

In [89]:
nb_model.predict_proba(X)[45].argmax()

39

In [90]:
df.iloc[45]

policy_id                                                      4187706
date_start                                         2020-07-18 00:00:00
country                                                    Afghanistan
type                                                  Health Resources
type_sub_cat                                               Ventilators
description          Afghanistan "on the 18th July, the Ministry of...
ISO_A3                                                             AFG
compliance                      Voluntary/Recommended but No Penalties
clean_description    afghanistan th july ministry public health sta...
encoded_sub-cat                                                    102
Name: 144, dtype: object

In [91]:
nb_model.predict_proba(X)[45][39]

0.15989831140973307

In [92]:
le.classes_[39]

'Masks'

## Well... got it wrong this time, but he gets it right for the first one (try urself)

How do I increase my score ? Need to do a nice grid search, maybe find a way to regroup some of the sub-cats, maybe only take the top 3 results, then REDO a naive Bayse pred from those => OVER THE WEEKEND